In [182]:
import pandas as pd
import numpy as np
import sklearn as sk

In [160]:
df = pd.read_csv('resume.csv')

In [161]:
df.shape

(2369, 20)

In [162]:
df.head(1)

,Unnamed: 0,age,auto,business_trips,categories,citizenship,city,education,employment,experience,experience_length,family,gender,languages,metro_or_moving,name,photo_url,salary,skills,url
0,0,20,NaN,не готов к командировкам,Архитектура,Россия,Ковров,"[{'kind': 'Неполное высшее образование', 'form...",неполный рабочий день,"[{'period': 'сентябрь 2017 – март 2018', 'leng...",7 месяцев,NaN,f,"['Немецкий язык, Базовый']",NaN,"3D-моделлер, IT стажёр, консультант",NaN,30 000₽,NaN,https://www.superjob.ru/resume/3d-modeller-436...


In [163]:
df.columns

Index(['Unnamed: 0', 'age', 'auto', 'business_trips', 'categories',
       'citizenship', 'city', 'education', 'employment', 'experience',
       'experience_length', 'family', 'gender', 'languages', 'metro_or_moving',
       'name', 'photo_url', 'salary', 'skills', 'url'],
      dtype='object')

In [164]:
df = df.drop('Unnamed: 0', axis=1)

In [165]:
df.auto.value_counts(dropna=False)

['B — легковые авто']                                                                                 1171
NaN                                                                                                    808
['B — легковые авто', 'C — грузовые авто']                                                             220
['A — мотоциклы', 'B — легковые авто']                                                                  74
['A — мотоциклы', 'B — легковые авто', 'C — грузовые авто']                                             52
['B — легковые авто', 'C — грузовые авто', 'D — автобусы']                                              11
['C — грузовые авто']                                                                                   11
['B — легковые авто', 'C — грузовые авто', 'D — автобусы', 'E — авто с прицепом']                        7
['B — легковые авто', 'D — автобусы']                                                                    5
['A — мотоциклы', 'B — легковые авто'

Много NaN, выкидываем.

In [166]:
df = df.drop('auto', axis=1)

In [167]:
df.business_trips.value_counts(dropna=False)

NaN                         1373
не готов к командировкам     968
на дому                       28
Name: business_trips, dtype: int64

Много NaN, выкидываем.

In [168]:
df = df.drop('business_trips', axis=1)

In [169]:
df.categories.value_counts(dropna=False)

Системное администрирование                      552
NaN                                              467
Техническая поддержка                            358
IT, Интернет, связь, телеком                     238
Инжиниринг                                       184
Администрирование баз данных                      80
Административная работа, секретариат, АХО         64
Сетевые технологии                                49
Разработка, программирование                      41
Передача данных и доступ в интернет               38
Управление проектами                              34
Аналитика                                         31
Интернет, создание и поддержка сайтов             23
Другое                                            18
Электронный документооборот                       15
Внедрение и сопровождение ПО                      12
Делопроизводство, ввод данных, систематизация     11
Компьютеры, оргтехника, ПО                         9
Web-верстка                                   

Выкинуть незаполненные (20 % данных) или весь признак? Неясно

In [170]:
df.citizenship.value_counts(dropna=False)

Россия               1615
NaN                   727
Украина                 8
Казахстан               7
Кыргызстан              3
Азербайджан             3
Беларусь                2
Молдова                 1
Дальнее зарубежье       1
Литва                   1
Таджикистан             1
Name: citizenship, dtype: int64

Хак: берем только Российское, считаем, что NaN - тоже Россия (можно это проверить по городам)

In [171]:
df = df[(df.citizenship == 'Россия')  | df.citizenship.isna()]
df = df.drop('citizenship', axis=1)

In [172]:
df.city.value_counts(dropna=False)

Москва                                 708
Санкт-Петербург                        285
Хабаровск                               64
Уфа                                     55
Иркутск                                 51
Ростов-на-Дону                          48
Самара                                  32
Краснодар                               32
Казань                                  27
Саратов                                 27
Рязань                                  26
Нижний Новгород                         24
Ульяновск                               23
Пенза                                   23
Волгоград                               22
Воронеж                                 20
Оренбург                                19
Сыктывкар                               17
Тверь                                   17
Смоленск                                16
Челябинск                               16
Екатеринбург                            16
Владимир                                16
Тюмень     

Обучать модель для регионов отдельно не хочется, будет слишком мало объектов. Предложение: выделить категории в зависимости от уровня зарплат по региону. Для начала 3: Мск, Спб, остальные.

In [207]:
ctgr = pd.get_dummies(df.city)
city_Msk = ctgr['Москва']
city_Spb = ctgr['Санкт-Петербург']
ctgr = ctgr.drop(['Москва', 'Санкт-Петербург'], axis=1)
city_oth = np.sum(ctgr.values, axis=1)
df['city_Msk'] = city_Msk
df['city_Spb'] = city_Spb
df['city_oth'] = city_oth
df = df.drop(['city'], axis=1)
df.shape

(2342, 18)

In [209]:
df.employment.value_counts(dropna=False)

полная занятость           1167
NaN                        1094
неполный рабочий день        33
сменный график работы        24
частичная занятость          13
работа вахтовым методом      11
Name: employment, dtype: int64

Много NaN, выкидываем.

In [211]:
df = df.drop(['employment'], axis=1)

In [221]:
df.family.value_counts(dropna=False)

cостоит в браке, есть дети       798
NaN                              682
не состоит в браке, детей нет    517
cостоит в браке, детей нет       166
не состоит в браке, есть дети    111
cостоит в браке                   36
не состоит в браке                27
есть дети                          3
детей нет                          2
Name: family, dtype: int64

In [222]:
df = df.drop('family', axis=1)

In [225]:
gender = pd.get_dummies(df.gender)
gender.f
df['gender'] = gender.f

In [229]:
df.languages.value_counts(dropna=False)

['Английский язык, Технический']                                                                                             829
NaN                                                                                                                          545
['Английский язык, Базовый']                                                                                                 459
['Английский язык, Разговорный']                                                                                             163
['Английский язык, Технический', 'Немецкий язык, Базовый']                                                                    47
['Английский язык, Базовый', 'Немецкий язык, Базовый']                                                                        42
['Английский язык, Cвободное владение']                                                                                       40
['Английский язык, Технический', 'Немецкий язык, Технический']                                   

In [230]:
df = df.drop('languages', axis=1)

In [231]:
df.metro_or_moving.value_counts(dropna=False)

NaN                                               1349
готов к переезду                                   532
Пражская                                            12
Улица Дыбенко                                        9
Комендантский проспект                               7
Проспект Ветеранов                                   6
Ботанический садВДНХ                                 6
Новогиреево                                          6
Коломенская                                          6
Выхино                                               6
Медведково                                           6
Первомайская                                         6
Планерная                                            5
Академическая                                        5
Новокосино                                           5
Проспект Большевиков                                 5
Щелковская                                           5
Саларьево                                            4
Митино    

In [232]:
df = df.drop('metro_or_moving', axis=1)

In [236]:
df.name.value_counts(dropna=False)

Системный администратор, IT-специалист                                                                         411
IT-специалист                                                                                                  203
IT специалист                                                                                                   76
ИТ-специалист                                                                                                   52
Руководитель IT-отдела                                                                                          36
Инженер IT                                                                                                      29
Руководитель отдела ИТ                                                                                          27
Руководитель IT отдела                                                                                          24
It-специалист                                                                   

Нужно обрабатывать как текст

In [237]:
df = df.drop('photo_url', axis=1)

In [240]:
df.skills.value_counts(dropna=False)

NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

Видимо, придется выкинуть незаполненные, важная инфа :(

In [243]:
df = df.drop('url', axis = 1)

In [245]:
df.shape

(2342, 12)

In [247]:
df.education.value_counts(dropna=False)

NaN                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    68
[{'kind': 'Среднее образование'}]                                                                                                                                                             

In [254]:
df = df[df.education.isna() == False]

Выкидываем незаполненные. Че дальше - непонятно

In [258]:
df.experience_length.value_counts(dropna=False)

NaN                     114
7 лет и 3 месяца         18
14 лет и 4 месяца        17
12 лет и 8 месяцев       15
11 лет и 4 месяца        15
10 лет и 6 месяцев       15
11 лет и 11 месяцев      15
9 лет и 7 месяцев        15
5 лет и 10 месяцев       14
10 лет и 2 месяца        14
7 лет и 2 месяца         14
2 года и 5 месяцев       13
6 лет и 6 месяцев        13
9 лет и 6 месяцев        13
3 года и 6 месяцев       13
7 лет и 8 месяцев        13
13 лет и 11 месяцев      13
10 лет и 3 месяца        13
13 лет и 2 месяца        12
16 лет и 5 месяцев       12
2 года и 10 месяцев      12
12 лет и 11 месяцев      12
12 лет и 5 месяцев       12
1 год и 3 месяца         12
16 лет и 2 месяца        12
9 лет и 11 месяцев       12
3 года и 8 месяцев       11
8 лет и 7 месяцев        11
11 лет и 6 месяцев       11
5 лет и 2 месяца         11
                       ... 
25 лет и 9 месяцев        1
37 лет и 2 месяца         1
34 года и 7 месяцев       1
35 лет и 5 месяцев        1
39 лет и 3 месяца   

In [259]:
df = df[df.experience.isna() == False]

In [263]:
df.shape

(2160, 12)

In [267]:
df.columns

Index(['age', 'categories', 'education', 'experience', 'experience_length',
       'gender', 'name', 'salary', 'skills', 'city_Msk', 'city_Spb',
       'city_oth'],
      dtype='object')

In [269]:
df.head(3)

,age,categories,education,experience,experience_length,gender,name,salary,skills,city_Msk,city_Spb,city_oth
0,20,Архитектура,"[{'kind': 'Неполное высшее образование', 'form...","[{'period': 'сентябрь 2017 – март 2018', 'leng...",7 месяцев,1,"3D-моделлер, IT стажёр, консультант",30 000₽,NaN,0,0,1
1,39,Техническая поддержка,"[{'kind': 'Высшее образование', 'form': 'Дневн...","[{'period': 'октябрь 2017 – работает сейчас', ...",18 лет,0,"Администратор, управляющий, IT специалист",50 000₽,[{'professional': '• 1С: Предприятие • 1С: Роз...,1,0,0
2,47,Передача данных и доступ в интернет,"[{'kind': 'Высшее образование', 'form': 'Вечер...","[{'period': 'февраль 2008 – работает сейчас', ...",12 лет и 10 месяцев,0,Администратор / Инженер технической поддержки IT,30 000₽,[{'professional': 'Английский язык уровень. Pr...,0,0,1


### Промежуточный итог: ###

Остались пропуски в 'categories' и 'skills', нужно решить, что с ними делать. В остальных фичах пропусков нет. 

Город и пол one-hot.

Что с образованием - неясно, очень много категорий.

Для 'experience' и 'skills' нужно посмотреть распределения N-грамм и выбрать хорошие, информативные фичи по вхождениям ключевых слов.

Так же необходимо все числовые признаки нормалтзовать.